# Robot localization using particle filter
The robot has steering and velocity control inputs. It has sensors that measures distance to visible landmarks. Both the sensors and control mechanism have noise in them, and we need to track the robot's position.

In [1]:
import copy
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from scipy import stats
from IPython.display import HTML
from ipywidgets import interactive
from matplotlib.patches import Circle

# use this command for the visulization of animation, uncomment otherwise
# %matplotlib notebook

# uncomment these two lines if you don't want multiple output in a cell
# just for the convenience of debugging
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Constuct particles
Particles can be constructed by randomly sampling in the 2D space, or by Gaussian sampling in place nearby.

In [2]:
def Uniform_Particles_Construction(width, height, N):

    # set the random seed so that we have reproducible experiments
    np.random.seed(500)

    particles = np.random.uniform([0,0], [width, height], size=(N, 2))
    return particles

In [3]:
def Gaussian_Particles_Construction(N, sigma):
    particles = sigma * np.random.randn(N, 2)
    return particles

## Reject sampled particles
Here we use the random sampling method as illustration. Since previously we get particles randomly among the total 2D spaces, it is possible that there are some particles located inside landmarks, thus we neet to delete these invalid particles.

In [4]:
def Rejection_Particles(particles, centers, radius):
    """Given randomly sampled particles and centers of landmarks, perform rejection here
    
    Args:
        particles: the particles we get through random generation in 2D space
        centers: centers of landmarks
        radius: the radius of cicular landmarks 
    """
    particles_after_rejection = []
    for count_p, coord_p in enumerate(particles):
        dis = np.linalg.norm(coord_p-centers, axis=1, keepdims=True)
        if np.all(dis >= radius):
            particles_after_rejection.append(particles[count_p])
    return np.asarray(particles_after_rejection)

Now let's predefine some parameters and then run two examples to see the influence of number of evidence. What do you see in these two examples?

In [5]:
[Width_Max, Height_Max]= [800, 600]

# Landmark center coordinates

# Version 1: 5 landmarks
Centers = np.array([ [336,175], [718,159], [510,43], [167, 333], [472, 437] ])
Radius=np.array([[12],[6],[7],[18],[9]])

# Version 2: ten landmarks
# Centers = np.array([ [144,73], [510,43], [336,175], [718,159], [178,484], [665,464], [267, 333], [541, 300], [472, 437], [100, 533] ])
# Radius=np.array([[12],[32],[7],[8],[13],[6],[7],[8],[9],[10]])

if len(Centers) != len(Radius):
    raise ValueError("Centers and Radius must have the same size!")

Num_Particles = np.arange(1000, 10000, 1000)

for i in range(len(Num_Particles)):
    particles = Uniform_Particles_Construction(Width_Max, Height_Max, Num_Particles[i])
    rejection =  Rejection_Particles(particles, Centers, Radius)
    print('Estimated acceptability for {} particles is {:.5f}'.format(Num_Particles[i], len(rejection[:,0])/len(particles[:,0])))

print('The true acceptability is {:.5f}'.format(1 - (np.pi * Radius * Radius).sum() / (Width_Max * Height_Max) ))

Estimated acceptability for 1000 particles is 0.99700
Estimated acceptability for 2000 particles is 0.99750
Estimated acceptability for 3000 particles is 0.99767
Estimated acceptability for 4000 particles is 0.99750
Estimated acceptability for 5000 particles is 0.99700
Estimated acceptability for 6000 particles is 0.99717
Estimated acceptability for 7000 particles is 0.99657
Estimated acceptability for 8000 particles is 0.99625
Estimated acceptability for 9000 particles is 0.99644
The true acceptability is 0.99585


## Motion model
Now we can move the remaining particles based on how you predict the real system is behaving with some noise in the motion model. Assuming that out sensors returns the data in a time interval of 0.5s.

In [16]:
def Predict(particles, v, std=1, dt=0.5):
    """Predict the motion of next state for each particles given current angles and velocities.
    
    Args:
        particles: the particles we get after rejecting the ones that are not available
        v： 2d array. Each sample with feature [angle, velocity]
        std: standard deviation of velociy, defaut 1
        dt: time interval, assume it to be 1 second here
    """
    N = len(particles)
    
    # add some noise to the distance
    # std can be set as a hyperparameter to decide how noisy is the data
    # thus we can change the difficulty and different version of the notebook
    delta_dist = (v[1] * dt) + (np.random.randn(N) * std)
    particles[:, 0] += np.cos(v[0]) * delta_dist
    particles[:, 1] += np.sin(v[0]) * delta_dist

## Update the weights of each particle
Update the weighting of the particles based on the measurement. Each particle has a position and a weight which estimates how well it matches the measurement. Normalizing the weights so they sum to one. This normalization step turns them into a probability distribution. Those particles that are closer to the robot will generally have a higher weight than ones far from the robot. Particles that closely match the measurements are weighted higher than particles which don't match the measurements very well. So in this case, we can measure the probability using the distance to landmarks.

In [17]:
# def Weights_Update(particles, weights, coord_rob, centers, radius, scale_fac, std):
def Weights_Update(particles, weights, dist_r_l, centers, radius, scale_fac, std):
    """Given the noised distances from robot to the landmarks, update the weights of particles
    
    Args:
        particles: coordinate of particles
        weights: weight of particles
        dist_r_l: the current distance between robot and landmarks
        scale_fac, std: hyperparameters to avoid the underflow of possibilities
    """
    
    weights.fill(1.)
    
    for count, center in enumerate(centers):
        # distance between the particles and each landmark
        dist_p_l = np.linalg.norm(particles-center, axis=1, keepdims=True) - radius[count]
        
        # have tried use exponential function to avoid underflow, but still of no use
        # since the distance is in the range of 1e2, so the possibility is 0 everywhere
        # so here use scale_fac and std to avoid the underflow of possibilities
        # set the distance as mean and std as standard deviation of norm distribution, then get the pdf as our new weights
        weights *= stats.norm.pdf(dist_p_l/scale_fac, dist_r_l[count]/scale_fac, std)

    weights += 1.e-100   # avoid round-off to zero
    weights /= sum(weights)

# Resample Procedures
Discard highly impossible particles and replace them with copies of the more possible particles. Accordingly, particles with greater weights survive with higher likelihood than particles with valus of small importance. In principle, there are many ways to achieve this, here you can refer to the procedure given in the paper [Resampling in Particle Filtering - Comparison](http://sait.cie.put.poznan.pl/38/SAIT_38_02.pdf) to complete the systematic resampling method, in which the algorithm is described as below (See Chapter.9, Page318):
<img src="img/systematic_resample.png" alt="Encoder" style="width: 400px;"/>
This resampling has a complexity of *O(N)*, and is one of the more readily recommended, because of its simplicity and operation speed. 

In [22]:
def systematic_resample(weights):
    
    Num_Weights = len(weights)
    
    # make N subdivisions, choose positions with a consistent random offset
    U = (np.arange(Num_Weights) + np.random.uniform()) / Num_Weights
 
    idx = np.zeros(Num_Weights, 'i') # set the data type as int
    sum_Q = np.cumsum(weights)
    
    i, j = 0, 0
    while i<Num_Weights and j<Num_Weights:
        if U[i] < sum_Q[j]:
            idx[i] = j
            i += 1
        else:
            j += 1
    return idx

The function above takes an array of weights and returns indexes of particles that have been chosen. We just need to write a function that performs the resampling from these indexes:

In [18]:
def resample_from_index(particles, weights, idx):
    particles[:] = particles[idx]
    weights[:] = weights[idx]
    weights /= np.sum(weights)

Now let's put the prediction positions of these particles together

# Load the data

In [32]:
# input the origin data as reference
Input_Sequence = np.load('./archive/Trajectory_1.npy')
# Now let's input the velocity and distance data
# So should be intepreted as corresponding transition and observability matrix in HMM type
Angle_Velocity = np.load('./data/velocity_1.npy')
Dist_r_l = np.load('./data/distance_1.npy')
# Now need to figure out what can be done next if you have velocity and diatance data at hand

In [33]:
np.shape(Input_Sequence)
np.shape(Angle_Velocity)
np.shape(Dist_r_l)

(37, 2)

(36, 2)

(36, 5, 1)

## Visulization
Now we visualize the moving of the robot to show how your particle filters works! What we need to achieve here is adding the moving of input sequence as well as particles onto that map.

In [37]:
# Then we get the particles
Random_Particles = Uniform_Particles_Construction(Width_Max, Height_Max, 500)
Reject_Random_Particles = Rejection_Particles(Random_Particles, Centers, Radius)
Origin_Random_Weights = np.ones((len(Reject_Random_Particles),1))

# Now we need to record the coordinates of moving particles
Prediction_Random_Paticles = [Reject_Random_Particles]

Random_Pos = copy.copy(Reject_Random_Particles)
Random_Weights = copy.copy(Origin_Random_Weights)

for i in range(len(Dist_r_l)):

    # The predicted position of particles
    Predict(Random_Pos, Angle_Velocity[i], 1, 0.5)
    Weights_Update(Random_Pos, Random_Weights, Dist_r_l[i], Centers, Radius, 50, 5)
    Random_Index = systematic_resample(Random_Weights)
    resample_from_index(Random_Pos, Random_Weights, Random_Index)
    Prediction_Random_Paticles.append(copy.copy(Random_Pos))
    
Particle_Random_Trajectory = np.asarray(Prediction_Random_Paticles)

In [38]:
def Location(step):
    img = plt.imread('img/Canvas.png')
    fig,ax = plt.subplots(1)

    # Now, loop through coord arrays, and create a circle at center
    for count, value in enumerate(Centers):
        circ = Circle(value,Radius[count])
        ax.add_patch(circ)
    
    ax.scatter(Input_Sequence[step,0], Input_Sequence[step,1], s=15, c='r', marker='x')
    ax.scatter(Particle_Random_Trajectory[step,:,0], Particle_Random_Trajectory[step,:,1],s=3, c='g')
    
    plt.xlim(0, Width_Max) 
    plt.ylim(0, Height_Max)

    # Create a figure. Equal aspect so circles look circular
    ax.set_aspect('equal')
    ax.imshow(img)

In [39]:
iplot = interactive(Location, step=(0, len(Particle_Random_Trajectory)-1))
iplot

interactive(children=(IntSlider(value=18, description='step', max=36), Output()), _dom_classes=('widget-intera…

We see that the robot starts at zero point, but the particles sampled randomly in the 2D spaces rarely go close to the zero, hence cause the bias which exists all over the trajectory. So instead, let's try with the Gaussian sample menthod.

In [43]:
Gaussian_Particles = Gaussian_Particles_Construction(500, 600)
Reject_Gaussian_Particles = Rejection_Particles(Gaussian_Particles, Centers, Radius)
Origin_Gaussian_Weights = np.ones((len(Reject_Gaussian_Particles),1))

# Now we need to record the coordinates of moving particles
Prediction_Gaussian_Paticles = [Reject_Gaussian_Particles]

Gaussian_Pos = copy.copy(Reject_Gaussian_Particles)
Gaussian_Weights = copy.copy(Origin_Gaussian_Weights)

for i in range(len(Input_Sequence)-1):

    Predict(Gaussian_Pos, Angle_Velocity[i], 1, 0.5)
    Weights_Update(Gaussian_Pos, Gaussian_Weights, Dist_r_l[i], Centers, Radius, 50, 5)
    Gaussian_Index = systematic_resample(Gaussian_Weights)
    resample_from_index(Gaussian_Pos, Gaussian_Weights, Gaussian_Index)
    Prediction_Gaussian_Paticles.append(copy.copy(Gaussian_Pos))
    
Particle_Gaussian_Trajectory = np.asarray(Prediction_Gaussian_Paticles)

In [44]:
def Location(step):
    img = plt.imread('img/Canvas.png')
    fig,ax = plt.subplots(1)

    # Now, loop through coord arrays, and create a circle at center
    for count, value in enumerate(Centers):
        circ = Circle(value,Radius[count])
        ax.add_patch(circ)
    
    ax.scatter(Input_Sequence[step,0], Input_Sequence[step,1], s=15, c='r', marker='x')
    ax.scatter(Particle_Gaussian_Trajectory[step,:,0], Particle_Gaussian_Trajectory[step,:,1],s=1, c='g')
    
    plt.xlim(0, Width_Max) 
    plt.ylim(0, Height_Max)

    # Create a figure. Equal aspect so circles look circular
    ax.set_aspect('equal')
    ax.imshow(img)

In [45]:
iplot = interactive(Location, step=(0, len(Particle_Gaussian_Trajectory)-1))
iplot

interactive(children=(IntSlider(value=18, description='step', max=36), Output()), _dom_classes=('widget-intera…

Then let's predict the position of robot's final position and make comparison with the true result. The final position should be somewhere around ...(tbd)

In [46]:
predict_final_random = np.average(Particle_Random_Trajectory[-1], weights=Random_Weights.flatten(), axis=0)
predict_final_random
predict_final_gaussian = np.average(Particle_Gaussian_Trajectory[-1], weights=Gaussian_Weights.flatten(), axis=0)
predict_final_gaussian
Input_Sequence[-1]

array([701.85080996, 260.75647265])

array([676.20904762, 248.06768738])

array([700.51813472, 253.59116022])